In [ ]:
import graph_generator as gg
import obm_dp as dp
import numpy as np
import matplotlib.pyplot as plt
from dp_analysis import NODE_FUNCS, expected_quantities

%load_ext autoreload
%autoreload 2

In [ ]:
config = {
    'graph_type': 'GEOM',
    'threshold': 0.0,
    'scaling': 1/ np.sqrt(2)
}

m = 5
n = 5
A = gg.sample_bipartite_graph(m, n, **config)
p = [1 for _ in range(m)]

In [ ]:
cache = dp.cache_stochastic_opt(A, p)
coin_flips = [np.random.binomial(1, _p) for _p in p]

In [ ]:
dp.stochastic_opt(A, coin_flips, cache)

In [ ]:
dp.greedy(A, coin_flips)

In [ ]:
def monte_carlo_layer_quantity(m: int, n: int, num_graphs: int, node_func: callable, **kwargs):
    data = []
    for _ in range(num_graphs):
        A = gg.sample_bipartite_graph(m, n, **kwargs)
        p = [0.75 for _ in range(m)]
        cache = {}
        cache = dp.cache_stochastic_opt(A, p)
        layers, quantities = expected_quantities(cache, node_func, A)
        data.append(quantities)
   
    return layers, np.array(data)


def plot_err(layers, means, errs, ylabel):
    plt.plot(layers, means)
    plt.fill_between(layers, means - errs, means + errs, alpha=0.3)
    plt.xlabel('DP Layer')
    plt.ylabel(ylabel)
    plt.show()


def analyze_dp(
    m: int,
    n: int,
    num_graphs: int,
    node_func: callable,
    ylabel: str,
    **graph_kwargs
) -> None:
    
    layers, data = monte_carlo_layer_quantity(m, n, num_graphs, node_func, **graph_kwargs)
    means = np.mean(data, axis=0)
    stds = np.std(data, axis=0, ddof=1)
    errs = 2 * stds / np.sqrt(num_graphs)  
    plot_err(layers, means, errs, ylabel)

    

In [ ]:
num_graphs = 100
node_func = NODE_FUNCS['greedy_suboptimality']
analyze_dp(m, n, num_graphs, node_func, 'Average fractional value-to-go reduction', **config)

In [ ]:
node_func = NODE_FUNCS['uniform_suboptimality']
analyze_dp(m, n, num_graphs, node_func, 'Average fractional value-to-go reduction', **config)

In [ ]:
node_func = NODE_FUNCS['greedy_correctness']
analyze_dp(m, n, num_graphs, node_func, 'Fraction of correct decisions', **config)

In [ ]:
node_func = NODE_FUNCS['relative_entropy']
analyze_dp(m, n, num_graphs, node_func, 'KL divergence with uniform', **config)

In [ ]:
config = {
    'graph_type': 'BA',
    'ba_param': 4,
    'weighted': True
}
m = 10; n = 7
num_graphs = 250
node_func = NODE_FUNCS['greedy_suboptimality']
analyze_dp(m, n, num_graphs, node_func, 'Average fractional value-to-go reduction', **config)

In [ ]:
num_graphs = 250
node_func = NODE_FUNCS['uniform_suboptimality']
analyze_dp(m, n, num_graphs, node_func, 'Average fractional value-to-go reduction', **config)

In [ ]:
node_func = NODE_FUNCS['greedy_correctness']
analyze_dp(m, n, num_graphs, node_func, 'Fraction of correct decisions', **config)